### Packages

In [52]:
import random
import string
import os

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding


### MonoAlphabetc Cipher

In [11]:
def shift_right(text, shift):
    result = ""
    for char in text:
        if char == '\n':
            result += char
        else:
            result += chr((ord(char) + shift) % 128)

    return result

def shift_left(text, shift):
    result = ""
    for char in text:
        if char == '\n':
            result += char
        else:
            result += chr((ord(char) - shift) % 128)

    return result


### Vignere Cipher

In [10]:
def vigenere_cipher(text):
    key = text[-5:]

    result = ""
    for i, char in enumerate(text):
        shift_amount = len(key) if i % 2 == 0 else -len(key)
        result += shift_right(char, shift_amount) if i % 2 == 0 else shift_left(char, shift_amount)

    return result, key

def vigenere_decipher(text, key):
    result = ""
    for i, char in enumerate(text):
        shift_amount = len(key) if i % 2 == 0 else -len(key)
        result += shift_left(char, shift_amount) if i % 2 == 0 else shift_right(char, shift_amount)

    return result

### Vernam Cipher

In [9]:

def generate_vernam_key(text_length):
    return ''.join(random.choice(string.ascii_letters + string.digits + string.punctuation) for _ in range(text_length))

def vernam_cipher(text, key):
    encrypted_text = ""
    for char, key_char in zip(text, key):
        encrypted_text += chr(ord(char) ^ ord(key_char))
    return encrypted_text

def vernam_decipher(encrypted_text, key):
    decrypted_text = ""
    for char, key_char in zip(encrypted_text, key):
        decrypted_text += chr(ord(char) ^ ord(key_char))
    return decrypted_text


### Transpositional Cipher

In [45]:
def transposition_encrypt(plaintext, key):
    padding = (key - len(plaintext) % key) % key
    plaintext += ' ' * padding

    transposition_matrix = [plaintext[i:i + key] for i in range(0, len(plaintext), key)]

    ciphertext = ''
    for col in range(key):
        ciphertext += ''.join(row[col] for row in transposition_matrix)

    return ciphertext

def transposition_decrypt(ciphertext, key):
    rows = len(ciphertext) // key

    transposition_matrix = [ciphertext[i:i + rows] for i in range(0, len(ciphertext), rows)]

    plaintext = ''
    for row in range(rows):
        plaintext += ''.join(col[row] for col in transposition_matrix)

    return plaintext.rstrip()


### RSA Cipher

In [27]:
def generate_keypair():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()

    # Serialize keys to PEM format
    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    return private_pem, public_pem

def rsa_encrypt_file(public_key_pem, input_file, output_file):
    with open(input_file, 'rb') as file:
        plaintext = file.read()

    public_key = serialization.load_pem_public_key(public_key_pem, backend=default_backend())
    ciphertext = public_key.encrypt(
        plaintext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    with open(output_file, 'wb') as file:
        file.write(ciphertext)

def rsa_decrypt_file(private_key_pem, input_file, output_file):
    with open(input_file, 'rb') as file:
        ciphertext = file.read()

    private_key = serialization.load_pem_private_key(private_key_pem, password=None, backend=default_backend())
    plaintext = private_key.decrypt(
        ciphertext,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    with open(output_file, 'wb') as file:
        file.write(plaintext)

### Encrypt and Decrypt File

In [41]:
shift_amount = 5
vkey = ''
vernkey = 0
num_columns = 5
private_key, public_key = generate_keypair()
tkey = 5


In [59]:
def encrypt_file(file_path, shift):
    global vkey, vernkey, public_key
    with open(file_path, 'r') as file:
        plaintext = file.read()
    
    
    vernkey = generate_vernam_key(len(plaintext))
    ciphertext1 = shift_right(plaintext, shift)
    ciphertext2, vkey = vigenere_cipher(ciphertext1)
    ciphertext3 = vernam_cipher(ciphertext2, vernkey)
    ciphertext4 = transposition_encrypt(ciphertext3 , tkey)

    with open('temp-encrypted.txt', 'w') as encrypted_file:
        encrypted_file.write(ciphertext4)


In [58]:
def decrypt_file(file_path, shift):
    global vkey, vernkey
    with open(file_path, 'r') as file:
        ciphertext = file.read()
        plaintext3 = transposition_decrypt(ciphertext, tkey)
        plaintext2 = vernam_decipher(plaintext3, vernkey)
        plaintext1 = vigenere_decipher(plaintext2, vkey)
        plaintext = shift_left(plaintext1, shift)

    with open('decrypted.txt', 'w') as decrypted_file:
        decrypted_file.write(plaintext)


### Main Function Calls

#### Encryption

In [61]:
file_path = 'example.txt'
encrypt_file(file_path, shift_amount)

rsa_encrypt_file(public_key, 'temp-encrypted.txt', 'encrypted.txt')
os.remove('temp-encrypted.txt')


#### Decryption

In [51]:
rsa_decrypt_file(private_key, 'encrypted.txt', 'temp-decrypted.txt' )

decrypt_file('temp-decrypted.txt', shift_amount)
os.remove('temp-decrypted.txt')
